DP vars: https://api.census.gov/data/2017/acs/acs5/profile/variables.html

All ACS5 vars: https://api.census.gov/data/2017/acs/acs5/variables.html

In [1]:
import geopandas  as gpd
import get_census as gc
import json
import os
import pandas as pd
from keys import census_api_key

In [2]:
# which vintage year
year = 2017

# which census dp variables to retrieve for each tract
dp_vars = ['DP05_0001E',    #total pop
           'DP04_0045E',    #occupied housing units
           'DP05_0077PE',   #pct pop non-hispanic white alone
           'DP02_0015E',    #avg household size
           'DP04_0007PE',   #pct single family detached homes
           'DP04_0089E',    #median value of owner occupied units (dollars)
           'DP04_0037E',    #median number of rooms in house
           'DP03_0062E',    #median household income (infl-adj 2017 usd)
           'DP02_0067PE',   #pct bachelor's degree or higher
           'DP03_0025E',    #mean travel time to work (minutes)
           'DP03_0019PE',   #pct commute drove alone
           'DP04_0026PE',   #1939 or earlier (pct housing structure built)
           'DP04_0025PE',   #1940-49
           'DP04_0024PE',   #1950-59
           'DP04_0023PE',   #1960-69
           'DP04_0022PE',   #1970-79
           'DP04_0021PE',   #1980-89
           'DP04_0020PE',   #1990-99
           'DP04_0019PE',   #2000-09
           'DP04_0018PE',   #2010-13
           'DP04_0017PE']   #2014 or later

# and pull anything not available from the dp separately and directly from the acs
acs_vars = ['B25046_001E']  #aggregate number of vehicles available (across all housing tenure types)

In [3]:
# get a list of all state fips codes
with open('data/states_by_fips.json') as f:
    states_by_fips = json.load(f)
non_states = ['American Samoa', 'Canal Zone', 'Guam', 'Puerto Rico', 'Virgin Islands']
states_fips = [k for k, v in states_by_fips.items() if v['name'] not in non_states]
len(states_fips)

51

In [4]:
%%time
# get the acs data, one state at a time, for all tracts in each state
df_acs = gc.download_census(api_key=census_api_key, dataset='acs/acs5', year=year, variables=acs_vars,
                            states=states_fips, clean=True, print_mode='state')

01 02 04 05 06 08 09 10 11 12 13 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 44 45 46 47 48 49 50 51 53 54 55 56 Wall time: 1min 4s


In [5]:
%%time
# get the acs dp data, one state at a time, for all tracts in each state
df_dp = gc.download_census(api_key=census_api_key, dataset='acs/acs5/profile', year=year, variables=dp_vars, 
                           states=states_fips, clean=True, print_mode='state')

01 02 04 05 06 08 09 10 11 12 Expecting value: line 1 column 1 (char 0) 500 There was an error while running your query.  We've logged the error and we'll correct it ASAP.  Sorry for the inconvenience. https://api.census.gov/data/2017/acs/acs5/profile?key=eb6ac6d6114c80e602f2599a8fe139d181a8e34f&get=DP05_0001E,DP04_0045E,DP05_0077PE,DP02_0015E,DP04_0007PE,DP04_0089E,DP04_0037E,DP03_0062E,DP02_0067PE,DP03_0025E,DP03_0019PE,DP04_0026PE,DP04_0025PE,DP04_0024PE,DP04_0023PE,DP04_0022PE,DP04_0021PE,DP04_0020PE,DP04_0019PE,DP04_0018PE,DP04_0017PE&for=tract:*&in=state:12 PAUSING THEN RE-TRYING
12 Expecting value: line 1 column 1 (char 0) 500 There was an error while running your query.  We've logged the error and we'll correct it ASAP.  Sorry for the inconvenience. https://api.census.gov/data/2017/acs/acs5/profile?key=eb6ac6d6114c80e602f2599a8fe139d181a8e34f&get=DP05_0001E,DP04_0045E,DP05_0077PE,DP02_0015E,DP04_0007PE,DP04_0089E,DP04_0037E,DP03_0062E,DP02_0067PE,DP03_0025E,DP03_0019PE,DP04_002

In [6]:
# map census vars to friendly names
cols = {'DP05_0001E'  : 'total_pop',
        'DP04_0045E'  : 'occupied_housing_units',
        'DP05_0077PE' : 'pct_white',
        'DP02_0015E'  : 'mean_household_size',
        'DP04_0007PE' : 'pct_single_fam',
        'DP04_0089E'  : 'med_home_value',
        'DP04_0037E'  : 'med_rooms_per_home',
        'DP04_0026PE' : 'pct_1939_earlier',
        'DP04_0025PE' : 'pct_1940_49',
        'DP04_0024PE' : 'pct_1950_59',
        'DP04_0023PE' : 'pct_1960_69',
        'DP04_0022PE' : 'pct_1970_79',
        'DP04_0021PE' : 'pct_1980_89',
        'DP04_0020PE' : 'pct_1990_99',
        'DP04_0019PE' : 'pct_2000_09',
        'DP04_0018PE' : 'pct_2010_13',
        'DP04_0017PE' : 'pct_2014_later',
        'DP03_0062E'  : 'med_hh_income',
        'DP03_0025E'  : 'mean_commute_time',
        'DP03_0019PE' : 'pct_drive_alone',
        'DP02_0067PE' : 'pct_bachelors_higher',
        'B25046_001E' : 'available_vehicles'}

# merge acs+dp dfs, rename columns, check lengths
df = pd.merge(left=df_dp, right=df_acs, how='inner', left_index=True, right_index=True).rename(columns=cols)
assert len(df) == len(df_acs) == len(df_dp)
len(df)

73056

In [7]:
# calculate vehicles per capita and per household
df['vehicles_per_capita'] = df['available_vehicles'] / df['total_pop']
df['vehicles_per_household'] = df['available_vehicles'] / df['occupied_housing_units']

In [8]:
# convert percents to proportions
for col in df.columns:
    if 'pct_' in col:
        new_col = col.replace('pct_', 'prop_')
        df[new_col] = df[col] / 100
        df = df.drop(columns=col)

In [9]:
# merge post-2010 structures-built columns into one column
df['prop_2010_later'] = df['prop_2010_13'] + df['prop_2014_later']
df = df.drop(columns=['prop_2010_13', 'prop_2014_later'])

In [10]:
# parse state and county into own columns
df['state'] = df.index.map(lambda x: x[0:2])
df['county'] = df.index.map(lambda x: x[2:5])

In [11]:
# view a sample row
df.iloc[1]

total_pop                     2172
occupied_housing_units         783
mean_household_size           2.52
med_home_value               96100
med_rooms_per_home             5.6
med_hh_income                41287
mean_commute_time             22.2
available_vehicles            1490
vehicles_per_capita       0.686004
vehicles_per_household     1.90294
prop_white                   0.416
prop_single_fam              0.759
prop_bachelors_higher        0.162
prop_drive_alone             0.905
prop_1939_earlier            0.144
prop_1940_49                 0.009
prop_1950_59                 0.047
prop_1960_69                 0.171
prop_1970_79                 0.325
prop_1980_89                 0.028
prop_1990_99                 0.117
prop_2000_09                 0.159
prop_2010_later                  0
state                           01
county                         001
Name: 01001020200, dtype: object

In [12]:
# save results to disk
df.to_csv('data/census_data.csv', index=True, encoding='utf-8')